In [11]:
import os
import numpy as np
import pandas as pd
from datetime import date

data_path = '/content/drive/MyDrive/Data Science/TAMIDS 2021/Competition/Data'

In [17]:
FEC_path = os.path.join(data_path, 'FEC_Clean')
files = os.listdir(FEC_path)
_path = lambda x: os.path.join(FEC_path, x)
files

['Linkages.csv',
 'Candidate Data.csv',
 'PAC Summary.csv',
 'Candidate Master.csv',
 'Committee Master.csv',
 'Committee to Committee.csv',
 'Committee to Candidates.csv',
 'Operation Expenses.csv']

In [3]:
print(files)
links = pd.read_csv(_path(files[0]))
cands = pd.read_csv(os.path.join(_path(files[1])))
pac = pd.read_csv(os.path.join(_path(files[2])))
cand_master = pd.read_csv(os.path.join(_path(files[3])))
ops = com_cand = pd.read_csv(os.path.join(_path(files[4])))
com_master = pd.read_csv(os.path.join(_path(files[5])))
com_com = pd.read_csv(os.path.join(_path(files[6])))
#induv_cand = pd.read_csv(os.path.join(_path(files[7])))
com_cand = pd.read_csv(os.path.join(_path(files[8])))

['Linkages.csv', 'Candidate Data.csv', 'PAC Summary.csv', 'Candidate Master.csv', 'Operation Expenses.csv', 'Committee Master.csv', 'Committee to Committee.csv', 'Induvidual Contributions.csv', 'Committee to Candidates.csv']


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6,10,11,12,15,16,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6,10,11,12,17,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
given_path = os.path.join(data_path, 'Given')
_path = lambda x: os.path.join(given_path, x)
votes = pd.read_csv(_path('countypres.csv'))
votes.head(2)

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2000,Alabama,AL,Autauga,1001.0,President,Al Gore,democrat,4942.0,17208,20191203
1,2000,Alabama,AL,Autauga,1001.0,President,George W. Bush,republican,11993.0,17208,20191203


In [18]:
name_ref = list(votes['candidate'].apply(lambda x: x.title()).unique()) + ['George Bush']
cand_master = pd.read_csv(os.path.join(_path('Candidate Master.csv')))


In [19]:
_names = cand_master['CAND_NAME'].apply(lambda x: " ".join(x.replace(',', '').split(' ')[:2][::-1]).title())
print(_names[:3].values, '....\n')

cand_master.loc[:,'CAND_NAME'] = _names.values

['Anna Young' 'Clifford Greene' 'Dae Miles'] ....



In [20]:
_keys = ['CAND_ID', 'CAND_NAME', 'CAND_PTY_AFFILIATION', 'CAND_ELECTION_YR', 'CAND_PCC']
pres_cands = cand_master.query(
    "CAND_OFFICE == 'P' &  CAND_PTY_AFFILIATION in ['DEM','REP'] \
    & CAND_STATUS != 'N' & CAND_NAME in @name_ref & CAND_ELECTION_YR >= 2000").copy()[_keys].drop_duplicates()
pres_cands.head(5)

,CAND_ID,CAND_NAME,CAND_PTY_AFFILIATION,CAND_ELECTION_YR,CAND_PCC
3516,P00003335,George Bush,REP,2000,C00360503
3810,P80000912,Al Gore,DEM,2000,C00360982
3833,P80002801,John Mccain,REP,2000,C00342154
7849,P80000235,John Kerry,DEM,2004,C00383653
12408,P00003335,George Bush,REP,2004,C00404343


In [22]:
p = '/content/drive/MyDrive/Data Science/TAMIDS 2021/Competition/Data/FEC_Processed/campain_ID.csv'

pres_cands.to_csv(p, encoding = 'utf-8', index = False)

In [ ]:
def fix_date(x):
  x = str(int(x))
  year = int(x[-4:])
  day = int(x[-6:-4])
  month = int(x[:-6])
  return date(year = year, month = month , day = day) 

x = com_com.query("CMTE_ID in @pres_cands.CAND_PCC | OTHER_ID in @pres_cands.CAND_PCC").dropna(
    subset = ['OTHER_ID', 'TRANSACTION_DT', 'ZIP_CODE'])

x = x.query("AMNDT_IND == 'N'")

x['datetime'] = x['TRANSACTION_DT'].apply(lambda x: fix_date(x))
x.loc[:, 'ZIP_CODE'] = x['ZIP_CODE'].apply(lambda x: int(float(x)))

In [ ]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

from scipy.stats import median_absolute_deviation as MAD

trans_types = x.groupby(['CMTE_ID', 'OTHER_ID', 'ZIP_CODE', 'datetime', 'TRANSACTION_TP'])['TRANSACTION_AMT'].agg(
    [len, np.sum, np.mean, np.std, np.median, MAD, percentile(.25), percentile(75)])
trans_types.sort_values(by = 'len', ascending = False, inplace = True)
transactions = trans_types.reset_index().rename(columns = {'len': 'count'})
transactions.fillna(0, inplace = True)

In [ ]:
good_cols = ['CMTE_ID', 'OTHER_ID', 'CITY', 'STATE', 'ZIP_CODE', 'datetime', 'TRANSACTION_TP', 'TRANSACTION_AMT']
com_com_clean = x[good_cols].reset_index(drop = True)
com_com_clean

,CMTE_ID,OTHER_ID,CITY,STATE,ZIP_CODE,datetime,TRANSACTION_TP,TRANSACTION_AMT
0,C00142943,C00342154,ALEXANDRIA,VA,22313,1999-06-02,24K,1000
1,C00142943,C00342154,ALEXANDRIA,VA,22313,1999-10-18,24K,1000
2,C00048371,C00342154,ALEXANDRIA,VA,22313,1999-11-11,24K,1000
3,C00236216,C00342154,ALEXANDRIA,VA,22313,1999-06-01,24K,1000
4,C00167460,C00342154,ALEXANDRIA,VA,22313,1999-11-12,24K,1000
...,...,...,...,...,...,...,...,...
125497,C00316455,C00580100,NEW YORK,NY,10022,2020-08-05,24K,2400
125498,C00607374,C00580100,NEW YORK,NY,10022,2020-08-26,24K,1000
125499,C00390161,C00580100,NEW YORK,NY,10022,2020-08-13,24K,-5000
125500,C00390161,C00580100,NEW YORK,NY,10022,2020-08-13,24K,-5000


In [ ]:
#new_file = os.path.join(data_path, 'FEC_Processed', 'com_com_transactions.csv')
#transactions.to_csv(new_file, encoding = 'utf-8')

#new_file = os.path.join(data_path, 'FEC_Processed', 'com_com_clean.csv')
#com_com_clean.to_csv(new_file, encoding = 'utf-8')

In [ ]:
com_cand = com_cand.query("CAND_ID in @pres_cands.CAND_ID & AMNDT_IND == 'N'").dropna(
    subset = ['OTHER_ID', 'TRANSACTION_DT', 'ZIP_CODE'])
com_cand['datetime'] = com_cand['TRANSACTION_DT'].apply(lambda x: fix_date(x))

In [ ]:
com_cand.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,datetime
47,C00311647,N,YE,P,2.003523e+10,24K,NaN,BUSH FOR PRESIDENT INC,AUSTIN,TX,78701,NaN,NaN,12081999.0,1000,C00343509,P00003335,NaN,NaN,NaN,CONTRIBUTION MADE TO NON-AFFILIATED,3062020110012558532,1999-12-08
215,C00142943,N,MY,P,9.902006e+10,24K,NaN,MCCAIN 2000 INC,ALEXANDRIA,VA,22313,NaN,NaN,6021999.0,1000,C00342154,P80002801,NaN,NaN,NaN,CONTRIBUTION MADE TO NON-AFFILIATED,3062020110009972461,1999-06-02
277,C00193342,N,YE,P,2.002003e+10,24K,NaN,BUSH FOR PRESIDENT INC,AUSTIN,TX,78701,NaN,NaN,9101999.0,1000,C00343509,P00003335,NaN,NaN,NaN,CONTRIBUTION MADE TO NON-AFFILIATED,3062020110012517390,1999-09-10
314,C00142943,N,YE,P,2.002001e+10,24K,NaN,MCCAIN 2000 INC,ALEXANDRIA,VA,22313,NaN,NaN,10181999.0,1000,C00342154,P80002801,NaN,NaN,NaN,CONTRIBUTION MADE TO NON-AFFILIATED,3062020110009725487,1999-10-18
487,C00345272,N,YE,P,2.003518e+10,24K,NaN,BUSH FOR PRESIDENT INC,AUSTIN,TX,78701,NaN,NaN,8171999.0,1000,C00343509,P00003335,NaN,NaN,NaN,CONTRIBUTION MADE TO NON-AFFILIATED,3062020110011470816,1999-08-17


In [ ]:
x = com_cand.groupby(
    ['CMTE_ID', 'OTHER_ID', 'TRANSACTION_TP', 'ZIP_CODE', 'datetime']
    )['TRANSACTION_AMT'].agg(
    [len, np.sum, np.mean, np.std, np.median, MAD, percentile(.25), percentile(75)])
x.sort_values(by = 'len', ascending = False, inplace = True)
contribs = x.reset_index().rename(columns = {'len': 'count'})

In [ ]:
#new_file = os.path.join(data_path, 'FEC_Processed', 'com_cand_transactions.csv')
#contribs.to_csv(new_file, encoding = 'utf-8')

com_cand_clean = com_cand[good_cols].reset_index(drop = True)
new_file = os.path.join(data_path, 'FEC_Processed', 'com_cand_clean.csv')
com_cand_clean.to_csv(new_file, encoding = 'utf-8')

In [ ]:
com_master = pd.read_csv(os.path.join(_path(files[5])))

dem_rep_cmtes = com_master.query("CMTE_PTY_AFFILIATION in ['DEM', 'REP']")[['CMTE_ID','CMTE_PTY_AFFILIATION']]

In [ ]:
induv_cand = pd.read_csv(os.path.join(_path(files[7])))

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5,6,7,11,13,16,17,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
def fix_date(x):
  x = str(int(x))
  year = int(x[-4:])
  day = int(x[-6:-4])
  month = int(x[:-6])
  return date(year = year, month = month , day = day) 

x = induv_cand.query("OTHER_ID in @pres_cands.CAND_PCC \
    | CMTE_ID in @dem_rep_cmtes ").dropna(
    subset = ['OTHER_ID', 'TRANSACTION_DT', 'ZIP_CODE'])
x['datetime'] = x['TRANSACTION_DT'].apply(lambda x: fix_date(x))

In [ ]:
good_cols = ['CMTE_ID', 'OTHER_ID', 'CITY', 'STATE', 'ZIP_CODE', 'datetime', 'TRANSACTION_TP', 'TRANSACTION_AMT']

x = x[good_cols].reset_index(drop = True)
x

,CMTE_ID,OTHER_ID,CITY,STATE,ZIP_CODE,datetime,TRANSACTION_TP,TRANSACTION_AMT
0,C00346098,C00383653,BOSTON,MA,2116,2003-03-12,24I,1000
1,C00121368,C00360982,HOUSTON,TX,77019,2003-12-31,24I,-200
2,C00346536,C00404343,DEMAREST,NJ,07627,2004-10-19,24I,1000
3,C00346536,C00404343,BELMONT BAY,VA,22191,2004-10-26,24I,2000
4,C00346536,C00404343,IRVING,TX,75062,2004-11-01,24I,200
...,...,...,...,...,...,...,...,...
4684,C00468314,C00431445,CINCINNATI,OH,45219,2012-10-14,24T,500
4685,C00468314,C00431445,LAGUNA BEACH,CA,92651,2012-10-16,24T,200
4686,C00468314,C00431445,LONG BEACH,CA,90803,2012-10-09,24T,200
4687,C00468314,C00431445,SACRAMENTO,CA,95816,2012-10-01,24T,250


In [ ]:
new_file = os.path.join(data_path, 'FEC_Processed', 'induv_com_clean.csv')
x.to_csv(new_file, encoding = 'utf-8', index = False)

In [ ]:
y = dem_rep_cmtes.reset_index(drop = True)
y

,CMTE_ID,CMTE_PTY_AFFILIATION
0,C00000125,REP
1,C00000935,DEM
2,C00000992,REP
3,C00001081,REP
4,C00001305,REP
...,...,...
45183,C00771394,REP
45184,C00771410,REP
45185,C00771436,DEM
45186,C00771451,REP


In [ ]:
new_file = os.path.join(data_path, 'FEC_Processed', 'com_party.csv')
y.to_csv(new_file, encoding = 'utf-8')